In [3]:
import pickle
import pandas as pd
from datetime import datetime
import os
import numpy as np

# Collection of all settings to run auxiliary tasks that we set up
aux_task_configs = {
    'none': {
        'auxiliary_tasks': [], #Inside env_kwargs
        'auxiliary_task_args': [], #Inside env_kwargs
        'auxiliary_heads': [], #Inside nn_base_kwargs
        'auxiliary_truth_sizes': [], #Inside config (parseargs)
        'aux_wrapper_kwargs': {}, #Inside config (parserags)
        'rew_structure': 'goal' #Inside env_kwargs
    },
    
    'rewexplore': {
        'auxiliary_tasks': [],
        'auxiliary_task_args': [],
        'auxiliary_heads': [],
        'auxiliary_truth_sizes': [],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'explore'
    },
    
    'rewdist': {
        'auxiliary_tasks': [],
        'auxiliary_task_args': [],
        'auxiliary_heads': [],
        'auxiliary_truth_sizes': [],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'dist'
    },
    
    'wall0': {
        'auxiliary_tasks': [2],
        'auxiliary_task_args': [0],
        'auxiliary_heads': [[-1, 0, 0, 1]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'wall1': {
        'auxiliary_tasks': [2],
        'auxiliary_task_args': [1],
        'auxiliary_heads': [[-1, 0, 0, 1]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'wall01': {
        'auxiliary_tasks': [2, 2],
        'auxiliary_task_args': [0, 1],
        'auxiliary_heads': [[-1, 0, 0, 1], [-1, 0, 0, 1]],
        'auxiliary_truth_sizes': [1, 1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'goaldist': {
        'auxiliary_tasks': [3],
        'auxiliary_task_args': [None],
        'auxiliary_heads': [[-1, 0, 0, 1]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'terminal': {
        'auxiliary_tasks': [],
        'auxiliary_task_args': [],
        'auxiliary_heads': [[-1, 0, 0, 1]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {'auxiliary_tasks': [0], 
                               'auxiliary_task_args': [None]},
        'rew_structure': 'goal'
    },
    
    'catwall0': {
        'auxiliary_tasks': [5],
        'auxiliary_task_args': [0],
        'auxiliary_heads': [[-1, 0, 1, 2]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'catwall0': {
        'auxiliary_tasks': [5],
        'auxiliary_task_args': [1],
        'auxiliary_heads': [[-1, 0, 1, 2]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'catwall01': {
        'auxiliary_tasks': [5, 5],
        'auxiliary_task_args': [0, 1],
        'auxiliary_heads': [[-1, 0, 1, 2], [-1, 0, 1, 2]],
        'auxiliary_truth_sizes': [1, 1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'catfacewall': {
        'auxiliary_tasks': [4],
        'auxiliary_task_args': [None],
        'auxiliary_heads': [[-1, 0, 1, 4]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
    
    'catquad': {
        'auxiliary_tasks': [6],
        'auxiliary_task_args': [None],
        'auxiliary_heads': [[-1, 0, 1, 4]],
        'auxiliary_truth_sizes': [1],
        'aux_wrapper_kwargs': {},
        'rew_structure': 'goal'
    },
}

# MWM North Poster Baseline

These were the experiments by which others were compared. Specifically batch size 16 and 32

In [ ]:
batch_sizes = [16, 32, 64]
num_trials = 3
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_pdistal_batch{batch}auxnone_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'goal',
                      'task_structure': 2, 'wall_colors': 1, 'num_rays': 12, 'fov': 1,
                      'poster': 1, 'character_reset_pos': 1}

        aux_heads = auxiliary_heads[n]
        aux_kwargs = aux_wrapper_kwargs[n]

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['layer_width'] = width
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 3000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            'checkpoint_interval': 10,
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            'nn_base': 'FlexBase',
            'nn_base_kwargs': {'hidden_size': width,
                               'auxiliary_heads': aux_heads},
            'aux_wrapper_kwargs': aux_kwargs
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))

# Testing one-hot observation encodings and retesting exploration MWM variation

## One-hot north poster

In [4]:
batch_sizes = [16, 32, 64]
num_trials = 5

rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_pnorth_batch{batch}onehot_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'goal',
                      'task_structure': 2, 'wall_colors': 1, 'num_rays': 12, 'fov': 1,
                      'poster': 1, 'character_reset_pos': 1,  'one_hot_obs': True}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 3000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            'checkpoint_interval': 10,
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            'nn_base': 'FlexBase',
            'nn_base_kwargs': {'hidden_size': 16},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,poster,character_reset_pos,one_hot_obs,name,batch_size
0,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehot_t0,16
1,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehot_t1,16
2,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehot_t2,16
3,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehot_t3,16
4,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehot_t4,16
5,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehot_t0,32
6,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehot_t1,32
7,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehot_t2,32
8,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehot_t3,32
9,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehot_t4,32


### No vecnormalize

In [12]:
batch_sizes = [16, 32, 64]
num_trials = 5

rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_pnorth_batch{batch}onehotnovec_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'goal',
                      'task_structure': 2, 'wall_colors': 1, 'num_rays': 12, 'fov': 1,
                      'poster': 1, 'character_reset_pos': 1,  'one_hot_obs': True}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 3000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            'checkpoint_interval': 10,
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            'nn_base': 'FlexBase',
            'nn_base_kwargs': {'hidden_size': 16},
            'normalize_env': False
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,poster,character_reset_pos,one_hot_obs,name,batch_size
0,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehotnovec_t0,16
1,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehotnovec_t1,16
2,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehotnovec_t2,16
3,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehotnovec_t3,16
4,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch16onehotnovec_t4,16
5,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehotnovec_t0,32
6,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehotnovec_t1,32
7,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehotnovec_t2,32
8,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehotnovec_t3,32
9,0,goal,2,1,12,1,1,1,True,nav_pnorth_batch32onehotnovec_t4,32


In [11]:
import sys
sys.path.append('../')
from scheduler import convert_config_to_command

convert_config_to_command('nav_pnorth_batch64onehotnovec_t0', config_folder='../experiment_configs/')

'python main.py --wandb-project-name Nav_Shared_Layers --exp-name nav_pnorth_batch64onehotnovec_t0 --save-name nav_pnorth_batch64onehotnovec_t0 --num-env-steps 3000000 --env-name NavEnv-v0 --algo ppo --num-mini-batch 1 --num-processes 1 --num-steps 64 --checkpoint-interval 10 --recurrent --seed 0 --no-cuda --env-kwargs num_objects=0 rew_structure=goal task_structure=2 wall_colors=1 num_rays=12 fov=1 poster=1 character_reset_pos=1 one_hot_obs=True --nn-base FlexBase --nn-base-kwargs hidden_size=16 --normalize-env False --config-file-name nav_pnorth_batch64onehotnovec_t0 '

# Explore env

In [5]:
batch_sizes = [16, 64, 128]
num_trials = 5

rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wallexplore_batch{batch}onehot_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'goal',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 3000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 1,
            'num_processes': 1,
            'num_steps': batch,
            'checkpoint_interval': 10,
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            'nn_base': 'FlexBase',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,name,batch_size
0,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch16onehot_t0,16
1,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch16onehot_t1,16
2,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch16onehot_t2,16
3,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch16onehot_t3,16
4,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch16onehot_t4,16
5,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch64onehot_t0,64
6,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch64onehot_t1,64
7,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch64onehot_t2,64
8,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch64onehot_t3,64
9,0,goal,3,4,12,1,1,True,nav_4wallexplore_batch64onehot_t4,64


## Tweaking some environment parameters

It appears from looking at trajectories that the agent can in fact explore, locate, and head back to the goal. So going to increase turn speed and move speed and number of steps per episode to encourage this even more

In [24]:
batch_sizes = [64, 128]
num_trials = 5

rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wallexplore_batch{batch}faster_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'goal',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True,
                      'turn_speed': 0.5, 'move_speed': 15, 'max_steps': 500}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 5000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            
            'checkpoint_interval': 10,
            'checkpoint_dir': '4wallexplore',
            'save_dir': '4wallexplore',
            
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            
            'nn_base': 'FlexBaseAux',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,name,batch_size
0,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster_t0,64
1,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster_t1,64
2,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster_t2,64
3,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster_t3,64
4,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster_t4,64
5,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster_t0,128
6,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster_t1,128
7,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster_t2,128
8,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster_t3,128
9,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster_t4,128


In [25]:
batch_sizes = [64, 128]
num_trials = 5

rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wallexplore_batch{batch}faster20m_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'goal',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True,
                      'turn_speed': 0.5, 'move_speed': 15, 'max_steps': 500}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 20000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            
            'checkpoint_interval': 10,
            'checkpoint_dir': '4wallexplore',
            'save_dir': '4wallexplore',
            
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            
            'nn_base': 'FlexBaseAux',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,name,batch_size
0,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster20m_t0,64
1,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster20m_t1,64
2,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster20m_t2,64
3,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster20m_t3,64
4,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch64faster20m_t4,64
5,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20m_t0,128
6,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20m_t1,128
7,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20m_t2,128
8,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20m_t3,128
9,0,goal,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20m_t4,128


In [3]:
batch_sizes = [128]
num_trials = 5

rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wallexplore_batch{batch}faster20mrewexplore_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'explore',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True,
                      'turn_speed': 0.5, 'move_speed': 10, 'max_steps': 500}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 20000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            
            'checkpoint_interval': 10,
            'checkpoint_dir': '4wallexplore',
            'save_dir': '4wallexplore',
            
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            
            'nn_base': 'FlexBaseAux',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,name,batch_size
0,0,explore,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20mrewexplore_t0,128
1,0,explore,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20mrewexplore_t1,128
2,0,explore,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20mrewexplore_t2,128
3,0,explore,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20mrewexplore_t3,128
4,0,explore,3,4,12,1,1,True,0.5,15,500,nav_4wallexplore_batch128faster20mrewexplore_t4,128


In [3]:
batch_sizes = [128]
num_trials = 5

rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wallexplore_batch{batch}explorepunish_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'explorepunish',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True,
                      'turn_speed': 0.5, 'move_speed': 10, 'max_steps': 500}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 5000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            
            'checkpoint_interval': 10,
            'checkpoint_dir': '4wallexplore',
            'save_dir': '4wallexplore',
            
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            
            'nn_base': 'FlexBaseAux',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,name,batch_size
0,0,explorepunish,3,4,12,1,1,True,0.5,10,500,nav_4wallexplore_batch128explorepunish_t0,128
1,0,explorepunish,3,4,12,1,1,True,0.5,10,500,nav_4wallexplore_batch128explorepunish_t1,128
2,0,explorepunish,3,4,12,1,1,True,0.5,10,500,nav_4wallexplore_batch128explorepunish_t2,128
3,0,explorepunish,3,4,12,1,1,True,0.5,10,500,nav_4wallexplore_batch128explorepunish_t3,128
4,0,explorepunish,3,4,12,1,1,True,0.5,10,500,nav_4wallexplore_batch128explorepunish_t4,128


In [6]:
sub_reward_sizes = [0.01, 0.02, 0.05, 0.1]
num_trials = 3

batch = 128
rows = []
for sub_reward in sub_reward_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wallexplore_ep{sub_reward}_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'explorepunish',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True,
                      'turn_speed': 0.5, 'move_speed': 10, 'max_steps': 500,
                      'sub_goal_reward': sub_reward}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 5000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            
            'checkpoint_interval': 10,
            'save_dir': '4we_punish',
            
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            
            'nn_base': 'FlexBaseAux',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,sub_goal_reward,name,batch_size
0,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wallexplore_ep0.01_t0,128
1,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wallexplore_ep0.01_t1,128
2,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wallexplore_ep0.01_t2,128
3,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wallexplore_ep0.02_t0,128
4,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wallexplore_ep0.02_t1,128
5,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wallexplore_ep0.02_t2,128
6,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wallexplore_ep0.05_t0,128
7,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wallexplore_ep0.05_t1,128
8,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wallexplore_ep0.05_t2,128
9,0,explorepunish,3,4,12,1,1,True,0.5,10,500,0.10,nav_4wallexplore_ep0.1_t0,128


In [3]:
sub_reward_sizes = [0.01, 0.02, 0.05, 0.1]
num_trials = 3

batch = 128
rows = []
for sub_reward in sub_reward_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wallexplore_2ep{sub_reward}_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'explorepunish2',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True,
                      'turn_speed': 0.5, 'move_speed': 10, 'max_steps': 500,
                      'sub_goal_reward': sub_reward}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 5000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            
            'checkpoint_interval': 10,
            'save_dir': '4we_punish',
            
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            
            'nn_base': 'FlexBaseAux',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,sub_goal_reward,name,batch_size
0,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wallexplore_2ep0.01_t0,128
1,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wallexplore_2ep0.01_t1,128
2,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wallexplore_2ep0.01_t2,128
3,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wallexplore_2ep0.02_t0,128
4,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wallexplore_2ep0.02_t1,128
5,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wallexplore_2ep0.02_t2,128
6,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wallexplore_2ep0.05_t0,128
7,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wallexplore_2ep0.05_t1,128
8,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wallexplore_2ep0.05_t2,128
9,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.10,nav_4wallexplore_2ep0.1_t0,128


In [6]:
sub_reward_sizes = [0.01, 0.02, 0.05, 0.1]
num_trials = 3

batch = 128
rows = []
for sub_reward in sub_reward_sizes:
    for i in range(num_trials):
        exp_name = f'nav_4wefc_2ep{sub_reward}_t{i}'

        env_kwargs = {'num_objects': 0, 'rew_structure': 'explorepunish2',
                      'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                      'character_reset_pos': 1,  'one_hot_obs': True,
                      'turn_speed': 0.5, 'move_speed': 10, 'max_steps': 500,
                      'sub_goal_reward': sub_reward}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 5000000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,
            
            'checkpoint_interval': 10,
            'save_dir': '4wefc_punish',
            
            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,
            
            'nn_base': 'DelayedRNNPPO',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))
        
pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,sub_goal_reward,name,batch_size
0,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wefc_2ep0.01_t0,128
1,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wefc_2ep0.01_t1,128
2,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.01,nav_4wefc_2ep0.01_t2,128
3,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wefc_2ep0.02_t0,128
4,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wefc_2ep0.02_t1,128
5,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.02,nav_4wefc_2ep0.02_t2,128
6,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wefc_2ep0.05_t0,128
7,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wefc_2ep0.05_t1,128
8,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.05,nav_4wefc_2ep0.05_t2,128
9,0,explorepunish2,3,4,12,1,1,True,0.5,10,500,0.10,nav_4wefc_2ep0.1_t0,128


In [8]:
sub_reward_sizes = [0.01, 0.02, 0.04]
punish_args = [3, 5, 7]
mults = [2, 5]
num_trials = 3

batch = 128
rows = []
for punish_arg in punish_args:
    for mult in mults:
        
        for sub_reward in sub_reward_sizes:
            for i in range(num_trials):
                exp_name = f'nav_4wefc_ep{sub_reward}mult{mult}arg{punish_arg}_t{i}'

                env_kwargs = {'num_objects': 0, 'rew_structure': 'explorepunish1_explorebonus',
                              'task_structure': 3, 'wall_colors': 4, 'num_rays': 12, 'fov': 1,
                              'character_reset_pos': 1,  'one_hot_obs': True,
                              'turn_speed': 0.5, 'move_speed': 10, 'max_steps': 500,
                              'sub_goal_reward': sub_reward, 'bonus_multiplier': mult,
                              'explore_punish_arg': punish_arg}

                row = env_kwargs.copy()
                row['name'] = exp_name
                row['batch_size'] = batch

                rows.append(row)
                config = {
                    'wandb_project_name': 'Nav_Shared_Layers',
                    'exp_name': exp_name,
                    'save_name': exp_name,
                    'num_env_steps': 5000000,
                    'env_name': 'NavEnv-v0',
                    'algo': 'ppo',
                    'num_mini_batch': 10,
                    'num_processes': 100,
                    'num_steps': batch,

                    'checkpoint_interval': 10,
                    'save_dir': '4wefc_punish',

                    'recurrent': True,
                    'seed': i,
                    'no_cuda': True,
                    'env_kwargs': env_kwargs,

                    'nn_base': 'DelayedRNNPPO',
                    'nn_base_kwargs': {'hidden_size': 64},
                }

                pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))

pd.DataFrame(rows)

,num_objects,rew_structure,task_structure,wall_colors,num_rays,fov,character_reset_pos,one_hot_obs,turn_speed,move_speed,max_steps,sub_goal_reward,bonus_multiplier,explore_punish_arg,name,batch_size
0,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.01,2,3,nav_4wefc_ep0.01mult2arg3_t0,128
1,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.01,2,3,nav_4wefc_ep0.01mult2arg3_t1,128
2,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.01,2,3,nav_4wefc_ep0.01mult2arg3_t2,128
3,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.02,2,3,nav_4wefc_ep0.02mult2arg3_t0,128
4,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.02,2,3,nav_4wefc_ep0.02mult2arg3_t1,128
5,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.02,2,3,nav_4wefc_ep0.02mult2arg3_t2,128
6,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.04,2,3,nav_4wefc_ep0.04mult2arg3_t0,128
7,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.04,2,3,nav_4wefc_ep0.04mult2arg3_t1,128
8,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.04,2,3,nav_4wefc_ep0.04mult2arg3_t2,128
9,0,explorepunish1_explorebonus,3,4,12,1,1,True,0.5,10,500,0.01,5,3,nav_4wefc_ep0.01mult5arg3_t0,128


# Giving hidden location information

In [5]:
num_trials = 3

batch = 128
rows = []
eps = [0.01, 0.02]
for ep in eps:
    for i in range(num_trials):
        exp_name = f'expl_goalpos_punishbonusep{ep}_t{i}'

        env_kwargs = {'rew_structure': 'explorepunish1_explorebonus',
                      'sub_goal_reward': ep, 'bonus_multiplier': 5,
                      'explore_punish_arg': 5, 'obs_set': 3}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 5000000,
            'env_name': 'ExploreNav-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,

            'checkpoint_interval': 10,
            'save_dir': 'explorenav_pos',

            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,

            'nn_base': 'DelayedRNNPPO',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))

pd.DataFrame(rows)

,rew_structure,sub_goal_reward,bonus_multiplier,explore_punish_arg,obs_set,name,batch_size
0,explorepunish1_explorebonus,0.01,5,5,3,expl_goalpos_punishbonusep0.01_t0,128
1,explorepunish1_explorebonus,0.01,5,5,3,expl_goalpos_punishbonusep0.01_t1,128
2,explorepunish1_explorebonus,0.01,5,5,3,expl_goalpos_punishbonusep0.01_t2,128
3,explorepunish1_explorebonus,0.02,5,5,3,expl_goalpos_punishbonusep0.02_t0,128
4,explorepunish1_explorebonus,0.02,5,5,3,expl_goalpos_punishbonusep0.02_t1,128
5,explorepunish1_explorebonus,0.02,5,5,3,expl_goalpos_punishbonusep0.02_t2,128


# Shortcut Nav

In [4]:
num_trials = 3
batch_sizes = [32, 64, 128]
rows = []
for batch in batch_sizes:
    for i in range(num_trials):
        exp_name = f'shortcutnav_shortcutprob0batch{batch}_t{i}'

        env_kwargs = {'shortcut_probability': 0}

        row = env_kwargs.copy()
        row['name'] = exp_name
        row['batch_size'] = batch

        rows.append(row)
        config = {
            'wandb_project_name': 'Nav_Shared_Layers',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 5000000,
            'env_name': 'ShortcutNav-v0',
            'algo': 'ppo',
            'num_mini_batch': 10,
            'num_processes': 100,
            'num_steps': batch,

            'checkpoint_interval': 10,
            'checkpoint_dir': 'shortcuts',
            'save_dir': 'shortcuts',

            'recurrent': True,
            'seed': i,
            'no_cuda': True,
            'env_kwargs': env_kwargs,

            'nn_base': 'FlexBaseAux',
            'nn_base_kwargs': {'hidden_size': 64},
        }

        pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))

    pd.DataFrame(rows)

In [9]:
num_trials = 3
batch_sizes = [32, 64, 128]
shortcut_probs = [0.1, 0.2, 0.4]
rows = []
for shortcut in shortcut_probs:
    for batch in batch_sizes:
        for i in range(num_trials):
            exp_name = f'shortcutnav_shortcutprob{shortcut}batch{batch}_t{i}'

            env_kwargs = {'shortcut_probability': shortcut}

            row = env_kwargs.copy()
            row['name'] = exp_name
            row['batch_size'] = batch

            rows.append(row)
            config = {
                'wandb_project_name': 'Nav_Shared_Layers',
                'exp_name': exp_name,
                'save_name': exp_name,
                'num_env_steps': 3000000,
                'env_name': 'ShortcutNav-v0',
                'algo': 'ppo',
                'num_mini_batch': 10,
                'num_processes': 100,
                'num_steps': batch,

                'checkpoint_interval': 10,
                'checkpoint_dir': 'shortcuts',
                'save_dir': 'shortcuts',

                'recurrent': True,
                'seed': i,
                'no_cuda': True,
                'env_kwargs': env_kwargs,

                'nn_base': 'FlexBaseAux',
                'nn_base_kwargs': {'hidden_size': 64},
            }

            pickle.dump(config, open('../experiment_configs/' + exp_name, 'wb'))

pd.DataFrame(rows)

,shortcut_probability,name,batch_size
0,0.1,shortcutnav_shortcutprob0.1batch32_t0,32
1,0.1,shortcutnav_shortcutprob0.1batch32_t1,32
2,0.1,shortcutnav_shortcutprob0.1batch32_t2,32
3,0.1,shortcutnav_shortcutprob0.1batch64_t0,64
4,0.1,shortcutnav_shortcutprob0.1batch64_t1,64
5,0.1,shortcutnav_shortcutprob0.1batch64_t2,64
6,0.1,shortcutnav_shortcutprob0.1batch128_t0,128
7,0.1,shortcutnav_shortcutprob0.1batch128_t1,128
8,0.1,shortcutnav_shortcutprob0.1batch128_t2,128
9,0.2,shortcutnav_shortcutprob0.2batch32_t0,32
